# Klasifikácia pomocou zložených modelov - Random Forests

Random Forests predstavujú v súčasnosti jeden z najpoužívanejších klasifikačných modelov. Sciki-learn obsahuje implementáciu tohoto algoritmu v triede `RandomForestClassifier`. Tento klasifikátor sa používa rovnakým spôsobom, ako ostatné klasifikátory.

Opäť načítame dáta ako v predošlých príkladoch. Tentoraz ale pre algoritmus Random Forests neodstránime redundantné atribúty a ponecháme aj nominálny atribút `Deck` (aj s chýbajúcimi hodnotami) a transformujeme ho pomocou One Hot Encoderu. 

In [1]:
# Titanic import a preprocessing

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

titanic = pd.read_csv("../data/titanic-processed.csv")

titanic = titanic.drop(columns=['ticket', 'cabin'])

titanic['sex'] = titanic['sex'].map({"male": 0, "female": 1})
titanic['has_family'] = titanic['has_family'].map({False: 0, True: 1})

titanic['fare_ordinal'] = titanic['fare_ordinal'].map({"normal": 0, "more expensive": 1, "most expensive": 2})
titanic['age_ordinal'] = titanic['age_ordinal'].map({"child": 0, "young": 1, "adult": 2, "old": 3}) 

titanic = pd.get_dummies(titanic, columns=['embarked', 'title_short', 'deck', 'title'])

titanic.head()

,pclass,survived,sex,age,sibsp,parch,fare,family,has_family,fare_ordinal,...,title_Master,title_Miss,title_Mlle,title_Mme,title_Mr,title_Mrs,title_Ms,title_Rev,title_Sir,title_the Countess
0,1,1,1,29.00,0,0,211.3375,0,0,2,...,0,1,0,0,0,0,0,0,0,0
1,1,1,0,0.92,1,2,151.5500,3,1,2,...,1,0,0,0,0,0,0,0,0,0
2,1,0,1,2.00,1,2,151.5500,3,1,2,...,0,1,0,0,0,0,0,0,0,0
3,1,0,0,30.00,1,2,151.5500,3,1,2,...,0,0,0,0,1,0,0,0,0,0
4,1,0,1,25.00,1,2,151.5500,3,1,2,...,0,0,0,0,0,1,0,0,0,0


Model môžeme upravovať viacerými parametrami. Keďže sa jedná o model, ktorý tvorí množstvo rôznych stromových klasifikátorov na rôznych podmnožinách vstupných dát, väčšina parametrov bude identická s rozhodovacími stromami: 
* `n_estimators` - počet stromov v "lese"
* `bootstrap` - 
* `oob_score` - True/False - či používať alebo nie out-of-bag príklady na odhad presnosti
* `criterion` - kritérium pre voľbu atribúrov - "gini", "entropy"
* `max_depth` - maximálna hĺbka stromu (ak je nastavená na None, expanduje sa plný strom)
* `min_samples_split` - najmenší počet príkladov potrebných pre vetvenie uzlu
* `min_samples_leaf` - najmenší možný počet príkladov v listovom uzle

In [2]:
X_titanic = titanic.drop('survived', axis=1) # vytvoríme maticu príznakov - použijeme všetky stĺpce okrem cieľového atribútu a uložíme do X_titanic
y_titanic = titanic['survived'] # vytvoríme vektor hodnôt cieľového atribútu ako stĺpec 'survived'

print(X_titanic.shape) # pre kontrolu môžeme vypísať rozmery matice hodnôt a vektora cieľového atribútu
print(y_titanic.shape)

from sklearn.model_selection import train_test_split # importujeme funkciu train_test_split()
X_train, X_test, y_train, y_test = train_test_split(X_titanic, y_titanic, test_size=0.3, random_state=1) # rozdelíme dataset do trénovacej a testovacej časti, tak že testovacia bude 30% z celkového datasetu

(1309, 43)
(1309,)


In [6]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix

rf = RandomForestClassifier(n_estimators=1000, max_depth=3)
rf.fit(X_train, y_train)

y_rf = rf.predict(X_test)

print(f"Presnosť (accuracy) modelu: {accuracy_score(y_test, y_rf)}")
cm = confusion_matrix(y_test, y_rf)
print(cm)

Presnosť (accuracy) modelu: 0.8040712468193384
[[203  33]
 [ 44 113]]


Dôležitosť atribútov je podstatný výstup z modelu Random Forest. K dôležitosti atribútov sa dostaneme pomocou `feature_importances_`, ktoré sú súčasťou modelu Random Forest. Môžeme ich usporiadať a vypísať aj s jednotlivými zodpovedajúcimi názvami atribútov.

In [4]:
sorted(zip(rf.feature_importances_, X_train.columns), reverse=True)

[(0.20687486865095067, 'age'),
 (0.20124169899257044, 'fare'),
 (0.1174875042654262, 'title_Mr'),
 (0.07629392638125317, 'title_short_Mr'),
 (0.057235216657540014, 'pclass'),
 (0.04084605874344792, 'family'),
 (0.04000732533384135, 'title_short_Mrs'),
 (0.03533934318370013, 'fare_ordinal'),
 (0.034434322857779774, 'sibsp'),
 (0.027376044714666127, 'sex'),
 (0.020314651390278188, 'parch'),
 (0.016173341768321057, 'embarked_Cherbourg'),
 (0.01443329550724064, 'age_ordinal'),
 (0.013980927875203488, 'embarked_Southampton'),
 (0.013931992205365057, 'deck_E'),
 (0.013114366205818611, 'title_Miss'),
 (0.010822562846998614, 'has_family'),
 (0.010785135719684593, 'deck_B'),
 (0.007319693526530305, 'deck_D'),
 (0.007226625302373765, 'deck_C'),
 (0.0053436617803156, 'embarked_Queenstown'),
 (0.005208517064428343, 'title_short_rare title'),
 (0.004650029361125913, 'deck_F'),
 (0.0046021707147210315, 'deck_A'),
 (0.0022622923188003125, 'title_Rev'),
 (0.00208278835912733, 'title_short_Miss'),
 (0.

#### Úloha 12.5.

Vyskúšajte natrénovať model Random Forests na dátach Titanic. 
Vyskúšajte rôzne hodnoty parametrov (najmä `n_estimators` nastaviť na rádovo rozdielne hodnoty, napr. 10, 100, 1000). 
Použite ale rôzne nastavenia parametrov stromov - porovnajte Random Forest s nastaveniami stromov, ktoré vyplynuli ako optimálne z predchádzajúcej úlohy. Porovnajte výsledky s Random Forest s takými stromami, ktoré obsahuju veľa plytkých stromov. Líšia sa nejako? 

In [ ]:
# YOUR CODE HERE